# 430031 用於資料處理的 Python 程式設計 – 評估二 B 部分
**姓名**：PoKai Huang  
**學號**：26254793  
**檔名**：Huang_PoKai_430031_AT2b.ipynb

本筆記本依作業要求完成：**載入 → 基礎檢視 → 型別修正 → 兩種迴圈（數值/字串）→ 摘要統計 → 缺失/重複值處理 →匯出**。  
> 各章節含「**商業解讀**」，更貼近實務分析。


## 目錄
1. **環境與讀檔**：1.1 關閉警告｜1.2 載入套件｜1.3 穩健讀檔（含例外）  
2. **資料載入驗證**：2.1 前/後五列｜2.2 形狀｜2.3 dtypes  
    - 商業解讀 A：資料覆蓋範圍與欄位直覺  
3. **資料清洗與型別修正**：3.1 欄名標準化｜3.2 日期/時間 → datetime｜3.3 文字→類別/數值｜3.4 型別變更報告  
    - 商業解讀 B：欄位語意與可用性  
4. **迴圈操作（作業指定）**：4.1 `for`：數值總和/平均｜4.2 `while`：字串轉小寫 → `*_lower`  
    - 商業解讀 C：量級與一致性  
5. **摘要統計（全面探索）**：5.1 describe｜5.2 自訂數值摘要｜5.3 類別 Top-10｜5.4 關聯洞察  
    - 商業解讀 D：趨勢、離群與關聯  
6. **資料品質檢查**：6.1 缺失值報表｜6.2 重複行/列 + 影響評估  
    - 商業解讀 E：品質對後續分析的影響  
7. **小結與匯出**


## 1. 環境與讀檔（Env & Read）  
**小項目**：1.1 關閉警告｜1.2 載入套件｜1.3 穩健讀檔（含例外處理）


In [5]:
# 1.1 關閉非致命警告
import warnings
warnings.filterwarnings("ignore")

# 1.2 載入套件
import pandas as pd
import numpy as np

# 1.3 穩健讀檔：優先 UTF-8，失敗則改用 latin-1；缺檔回報清楚訊息。
DATA_PATH = "domestic_industrial_price_index.csv"

def safe_read_csv(path: str) -> pd.DataFrame:
    try:
        df = pd.read_csv(path)
        print(f"[INFO] 已成功載入：{path}")
        return df
    except UnicodeDecodeError:
        print("[WARN] 編碼非 UTF-8，改用 latin-1 嘗試…")
        return pd.read_csv(path, encoding="latin-1")
    except FileNotFoundError as e:
        raise FileNotFoundError(f"[ERROR] 找不到檔案：{path}，請確認 DATA_PATH 或將 CSV 放同資料夾。") from e

df = safe_read_csv(DATA_PATH)
assert isinstance(df, pd.DataFrame), "[FATAL] 讀檔失敗或資料不是 DataFrame"


[INFO] 已成功載入：domestic_industrial_price_index.csv


## 2. 資料載入驗證（Load Verification）  
**小項目**：2.1 前/後五列｜2.2 形狀｜2.3 dtypes


In [4]:
# 2.1 前五列 / 後五列
display(df.head(5))
display(df.tail(5))

# 2.2 形狀
print("\n[INFO] 形狀 (rows, cols):", df.shape)

# 2.3 欄位型別
print("\n[INFO] 各欄位型別：")
print(df.dtypes)

,STATISTIC,Statistic Label,TLIST(M1),Month,C02596V03150,Industry Sector NACE Rev 2,UNIT,VALUE
0,WPM40C01,Domestic Industrial Price Index (Excl VAT),202101,2021 January,10,Food products (10),Base 2021=100,99.046935
1,WPM40C01,Domestic Industrial Price Index (Excl VAT),202101,2021 January,11,Manufacture of beverages (11),Base 2021=100,NaN
2,WPM40C01,Domestic Industrial Price Index (Excl VAT),202101,2021 January,12,Tobacco products (12),Base 2021=100,NaN
3,WPM40C01,Domestic Industrial Price Index (Excl VAT),202101,2021 January,13,Textiles (13),Base 2021=100,98.441859
4,WPM40C01,Domestic Industrial Price Index (Excl VAT),202101,2021 January,14,Wearing apparel (14),Base 2021=100,99.771523


,STATISTIC,Statistic Label,TLIST(M1),Month,C02596V03150,Industry Sector NACE Rev 2,UNIT,VALUE
2295,WPM40C02,Export Industrial Price Index (Excl VAT),202410,2024 October,30,Other transport equipment (30),Base 2021=100,NaN
2296,WPM40C02,Export Industrial Price Index (Excl VAT),202410,2024 October,31,Furniture (31),Base 2021=100,111.9
2297,WPM40C02,Export Industrial Price Index (Excl VAT),202410,2024 October,32,Other manufacturing (32),Base 2021=100,NaN
2298,WPM40C02,Export Industrial Price Index (Excl VAT),202410,2024 October,V0800,Mining and quarrying (05 to 09),Base 2021=100,105.4
2299,WPM40C02,Export Industrial Price Index (Excl VAT),202410,2024 October,33,Repair and installation of machinery and equip...,Base 2021=100,NaN



[INFO] 形狀 (rows, cols): (2300, 8)

[INFO] 各欄位型別：
STATISTIC                      object
Statistic Label                object
TLIST(M1)                       int64
Month                          object
C02596V03150                   object
Industry Sector NACE Rev 2     object
UNIT                           object
VALUE                         float64
dtype: object


### 商業解讀 A（初步）
- 先確認**時間欄位**與**數值指數欄位**是否存在，作為趨勢/季節性分析的前提。  
- 若有**類別欄**（如 Domestic/Industrial/區域），可規劃分群比較的切片策略。  
- 讓後續方法（摘要統計、相關性）有明確的欄位對應。


## 3. 資料清洗與型別修正（Cleaning & Dtypes）  
**小項目**：3.1 欄名標準化｜3.2 日期→datetime｜3.3 文字→類別/數值｜3.4 變更報告


In [6]:
df_clean = df.copy()

# 3.1 欄名標準化
old_cols = df_clean.columns.tolist()
df_clean.columns = (
    df_clean.columns
      .str.strip()
      .str.lower()
      .str.replace(" ", "_")
)
col_map = dict(zip(old_cols, df_clean.columns))
print("[INFO] 欄名標準化完成（原→新）前幾個：", list(col_map.items())[:8])

# 3.2 / 3.3 型別修正
dtype_changes = {}
for col in df_clean.columns:
    old_dtype = df_clean[col].dtype
    
    # 日期/時間欄
    if ("date" in col) or ("time" in col):
        df_clean[col] = pd.to_datetime(df_clean[col], errors="coerce")
    
    # 文字欄：category 或 numeric
    if df_clean[col].dtype == "object":
        nunique_ratio = df_clean[col].nunique(dropna=True) / max(len(df_clean), 1)
        if nunique_ratio <= 0.30:
            df_clean[col] = df_clean[col].astype("category")
        else:
            df_clean[col] = pd.to_numeric(df_clean[col], errors="ignore")
    
    new_dtype = df_clean[col].dtype
    if new_dtype != old_dtype:
        dtype_changes[col] = (old_dtype, new_dtype)

print("\n[INFO] 型別修正後 dtypes：")
print(df_clean.dtypes)

# 3.4 型別變更報告
if dtype_changes:
    print("\n[REPORT] 型別變更清單：")
    for k, v in dtype_changes.items():
        print(f"  - {k}: {v[0]} -> {v[1]}")
else:
    print("\n[REPORT] 無型別變更。")

[INFO] 欄名標準化完成（原→新）前幾個： [('STATISTIC', 'statistic'), ('Statistic Label', 'statistic_label'), ('TLIST(M1)', 'tlist(m1)'), ('Month', 'month'), ('C02596V03150', 'c02596v03150'), ('Industry Sector NACE Rev 2', 'industry_sector_nace_rev_2'), ('UNIT', 'unit'), ('VALUE', 'value')]

[INFO] 型別修正後 dtypes：
statistic                     category
statistic_label               category
tlist(m1)                        int64
month                         category
c02596v03150                  category
industry_sector_nace_rev_2    category
unit                          category
value                          float64
dtype: object

[REPORT] 型別變更清單：
  - statistic: object -> category
  - statistic_label: object -> category
  - month: object -> category
  - c02596v03150: object -> category
  - industry_sector_nace_rev_2: object -> category
  - unit: object -> category


### 商業解讀 B（欄位可用性）
- 標準化欄名避免大小寫與空白問題；日期轉 `datetime` 以支援**時間序列**。  
- 低基數字串轉 `category` 有助節省記憶體；可轉數值則能進行**統計/建模**。  
- 型別變更報告讓後續結果**可追溯**，提升可讀性與信任度。


## 4. 迴圈操作（Loops；作業指定）  
**小項目**：4.1 for 數值總和/平均｜4.2 while 字串轉小寫 → 新欄


In [ ]:
numeric_cols = df_clean.select_dtypes(include=[np.number]).columns.tolist()
string_like_cols = df_clean.select_dtypes(include=["object", "category"]).columns.tolist()

# 4. 範例：使用 for / while 迴圈    
NUM_COL = numeric_cols[0] if numeric_cols else None
STR_COL = string_like_cols[0] if string_like_cols else None

print(f"[選用數值欄] {NUM_COL}")
print(f"[選用字串/類別欄] {STR_COL}")

# 4.1 for：總和與平均（手動）
if NUM_COL:
    total = 0.0
    count = 0
    for v in df_clean[NUM_COL]:
        if pd.notna(v):
            total += float(v)
            count += 1
    mean_val = total / count if count else np.nan
    print(f"[LOOP] `{NUM_COL}` 總和 = {total:.4f}；平均 = {mean_val:.4f}")
else:
    print("[WARN] 未找到數值欄，略過 for 示範。")

# 4.2 while：字串轉小寫 → 新欄
if STR_COL:
    new_col = f"{STR_COL}_lower"
    df_clean[new_col] = df_clean[STR_COL].astype(str)
    i = 0
    while i < len(df_clean):
        val = df_clean.at[i, STR_COL]
        df_clean.at[i, new_col] = str(val).lower() if pd.notna(val) else val
        i += 1
    print(f"[LOOP] 已新增小寫欄位：{new_col}")
    display(df_clean[[STR_COL, new_col]].head(10))
else:
    print("[WARN] 未找到字串/類別欄，略過 while 示範。")

[選用數值欄] tlist(m1)
[選用字串/類別欄] statistic
[LOOP] `tlist(m1)` 總和 = 465174450.0000；平均 = 202249.7609
[LOOP] 已新增小寫欄位：statistic_lower


,statistic,statistic_lower
0,WPM40C01,wpm40c01
1,WPM40C01,wpm40c01
2,WPM40C01,wpm40c01
3,WPM40C01,wpm40c01
4,WPM40C01,wpm40c01
5,WPM40C01,wpm40c01
6,WPM40C01,wpm40c01
7,WPM40C01,wpm40c01
8,WPM40C01,wpm40c01
9,WPM40C01,wpm40c01


### 商業解讀 C（規格與一致性）
- 先掌握數值欄的**量級**與**平均水平**，避免後續模型被極端值誤導。  
- 將字串規整為小寫可避免**同義不同形**造成分組錯誤（如 Domestic vs domestic）。


## 5. 摘要統計（Summary Statistics）  
**小項目**：5.1 describe｜5.2 自訂數值摘要｜5.3 類別 Top-10｜5.4 關聯洞察


In [12]:
# 5.1 describe（含非數值）
display(df_clean.describe(include="all").T)

# 5.2 數值欄自訂摘要
num_df = df_clean.select_dtypes(include=[np.number])
if not num_df.empty:
    summary_tbl = pd.DataFrame({
        "count": num_df.count(),
        "mean": num_df.mean(),
        "median": num_df.median(),
        "std": num_df.std(ddof=1),
        "min": num_df.min(),
        "q1": num_df.quantile(0.25),
        "q3": num_df.quantile(0.75),
        "max": num_df.max()
    })
    display(summary_tbl)
else:
    print("[WARN] 無數值欄位可生成摘要統計。")

# 5.3 類別欄 Top-10 次數
cat_cols = df_clean.select_dtypes(include=["category", "object"]).columns.tolist()
for col in cat_cols:
    print(f"\n[CAT FREQ] {col}（Top-10）")
    display(df_clean[col].value_counts(dropna=False).head(10))

# 5.4 數值欄關聯洞察：|r| Top-5
if not num_df.empty and num_df.shape[1] >= 2:
    corr = num_df.corr(numeric_only=True)
    corr_pairs = (
        corr.where(np.triu(np.ones(corr.shape), k=1).astype(bool))
            .stack()
            .abs()
            .sort_values(ascending=False)
    )
    top5 = corr_pairs.head(5)
    print("\n[INSIGHT] 數值欄相關係數（|r| Top-5）：")
    for (c1, c2), val in top5.items():
        print(f"  - {c1} vs {c2}: |r| = {val:.3f}")
else:
    print("\n[INSIGHT] 數值欄不足以計算兩兩相關。")
    

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
statistic,2300,2,WPM40C01,1150,NaN,NaN,NaN,NaN,NaN,NaN,NaN
statistic_label,2300,2,Domestic Industrial Price Index (Excl VAT),1150,NaN,NaN,NaN,NaN,NaN,NaN,NaN
tlist(m1),2300.0,NaN,NaN,NaN,202249.76087,109.412641,202101.0,202112.0,202211.5,202311.0,202410.0
month,2300,46,2021 April,50,NaN,NaN,NaN,NaN,NaN,NaN,NaN
c02596v03150,2300,25,10,92,NaN,NaN,NaN,NaN,NaN,NaN,NaN
industry_sector_nace_rev_2,2300,25,Basic metals (24),92,NaN,NaN,NaN,NaN,NaN,NaN,NaN
unit,2300,1,Base 2021=100,2300,NaN,NaN,NaN,NaN,NaN,NaN,NaN
value,1288.0,NaN,NaN,NaN,108.881253,11.241957,78.720396,100.539223,105.957636,113.498125,160.242641
statistic_lower,2300,2,wpm40c01,1150,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,count,mean,median,std,min,q1,q3,max
tlist(m1),2300,202249.760870,202211.500000,109.412641,202101.000000,202112.000000,202311.000000,202410.000000
value,1288,108.881253,105.957636,11.241957,78.720396,100.539223,113.498125,160.242641



[CAT FREQ] statistic（Top-10）


statistic
WPM40C01    1150
WPM40C02    1150
Name: count, dtype: int64


[CAT FREQ] statistic_label（Top-10）


...


[CAT FREQ] unit（Top-10）


unit
Base 2021=100    2300
Name: count, dtype: int64


[CAT FREQ] statistic_lower（Top-10）


statistic_lower
wpm40c01    1150
wpm40c02    1150
Name: count, dtype: int64


[INSIGHT] 數值欄相關係數（|r| Top-5）：
  - tlist(m1) vs value: |r| = 0.403


### 商業解讀 D（趨勢與關聯）
- **mean vs median** 可檢查偏態；**Q1/Q3** 反映波動範圍（IQR）。  
- 類別 Top-10 可看出資料主要集中類別；有助決定**分群視角**。  
- 高 |r| 的數值欄代表**聯動性強**，後續可作為共同指標或簡易預測的依據。


## 6. 資料品質檢查（Data Quality）  
**小項目**：6.1 缺失值報表｜6.2 重複行/列 + 影響評估


In [6]:
# 6.1 缺失值報表
na_count = df_clean.isna().sum()
na_pct = (na_count / len(df_clean) * 100).round(2) if len(df_clean) else na_count

null_report = pd.DataFrame({
    "null_count": na_count,
    "null_pct(%)": na_pct
}).sort_values(by="null_count", ascending=False)

display(null_report)

# 6.2 重複值偵測與處理
orig_shape = df_clean.shape

# 行重複
dup_rows_mask = df_clean.duplicated(keep="first")
dup_rows = dup_rows_mask.sum()
print(f"[INFO] 重複行數量：{dup_rows}")
if dup_rows > 0:
    df_clean = df_clean.loc[~dup_rows_mask].copy()
    print("[ACTION] 已移除重複行；新形狀：", df_clean.shape)

# 列重複（內容完全相同）
dup_cols_mask = df_clean.T.duplicated(keep="first")
dup_cols = df_clean.columns[dup_cols_mask].tolist()
print(f"[INFO] 重複列數量：{len(dup_cols)}")
if dup_cols:
    print("[INFO] 重複列清單：", dup_cols)
    df_clean = df_clean.drop(columns=dup_cols)
    print("[ACTION] 已移除重複列；新形狀：", df_clean.shape)

after_shape = df_clean.shape
print(f"[REPORT] 形狀變化：{orig_shape} -> {after_shape}")

,null_count,null_pct(%)
value,1012,44.0
statistic,0,0.0
statistic_label,0,0.0
month,0,0.0
tlist(m1),0,0.0
c02596v03150,0,0.0
industry_sector_nace_rev_2,0,0.0
unit,0,0.0
statistic_lower,0,0.0


[INFO] 重複行數量：0
[INFO] 重複列數量：0
[REPORT] 形狀變化：(2300, 9) -> (2300, 9)


### 商業解讀 E（品質影響）
- 高缺失率欄位不宜直接用於模型或決策；需考慮**補值策略**（如前值/中位數）。  
- 移除重複行/列可避免統計偏移與維度冗餘，提升**結果可信度與效率**。  
- 形狀變化反映「**可用樣本量**」的淨值，應在報告中說明。


## 7. 小結與匯出
- 已完成：載入 / 檢視 / 型別修正 / 迴圈示範 / 摘要統計 / 缺失與重複值處理。  
- 匯出清理後資料供後續分析或複測。


In [7]:
OUTPUT_PATH = "price_index_cleaned.csv"
try:
    df_clean.to_csv(OUTPUT_PATH, index=False)
    print(f"[INFO] 已匯出清理後資料：{OUTPUT_PATH}")
except Exception as e:
    print(f"[WARN] 匯出失敗：{e}")

[INFO] 已匯出清理後資料：price_index_cleaned.csv
